In [15]:
import pandas as pd
import json
import re
import plotly.express as px
import numpy as np
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

### RQ2.5
varying the values (context) of the parameters in the then clause

In [16]:
current_result = "5_shared_scenarios_results"

In [17]:
file_path = f'res/outcome/similarities/{current_result}.csv'
df = pd.read_csv(file_path, sep=';')

In [18]:
df.drop(columns=['Desired When DNF',
       'Shared When DNF', 'Penality When', 'Global Similarity When',
       'Final Similarity When',"Desired Given DNF","Shared Given DNF","Penality Given","Global Similarity Given","Final Similarity Given"], inplace=True)
df

,Scenario,Desired Then DNF,Shared Then DNF,Penality Then,Global Similarity Then,Final Similarity Then,Scenario Similarity,Action
0,Candidate_1,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,0.0,1.000000,1.000000,1.000000,behavior_1
1,Candidate_23,height >= 30 AND source_distance == 0,height >= 25 AND source_distance == 0,0.0,0.966665,0.966665,0.988888,behavior_23
2,Candidate_34,height >= 30 AND source_distance == 0,height >= 50 AND source_distance == 0,0.0,0.857145,0.857145,0.952382,behavior_34
3,Candidate_122,height >= 30 AND source_distance == 0,height <= 100 AND source_distance == 0,0.0,0.850000,0.850000,0.950000,behavior_122
4,Candidate_12,height >= 30 AND source_distance == 0,height >= 0 AND source_distance == 0,0.0,0.850000,0.850000,0.950000,behavior_12
...,...,...,...,...,...,...,...,...
127,Candidate_93,height >= 30 AND source_distance == 0,height <= 25 AND source_distance >= 75,0.0,0.000000,0.000000,0.666667,behavior_93
128,Candidate_94,height >= 30 AND source_distance == 0,height <= 25 AND source_distance >= 100,0.0,0.000000,0.000000,0.666667,behavior_94
129,Candidate_95,height >= 30 AND source_distance == 0,height <= 25 AND source_distance <= 0,0.0,0.000000,0.000000,0.666667,behavior_95
130,Candidate_60,height >= 30 AND source_distance == 0,height >= 100 AND source_distance >= 75,0.0,0.000000,0.000000,0.666667,behavior_60


In [19]:
def extract_clauses(expression):
    # Primeiro, dividir a expressão usando 'AND' como delimitador
    clauses = expression.split(' AND ')
    
    # Regex para capturar o padrão: variável, operador, e valor, incluindo == e !=
    pattern = r'(\w+)\s*(>=|<=|>|<|=|==|!=)\s*([\d.]+)'
    
    # Extrair e converter as cláusulas em um dicionário
    result = {}
    for clause in clauses:
        match = re.match(pattern, clause.strip())
        if match:
            var, op, val = match.groups()
            val = float(val)
            result[var] = (var, op, val)
    
    return result


def create_pairs(row):
    clauses_1 = extract_clauses(row['Desired Then DNF'])
    clauses_2 = extract_clauses(row['Shared Then DNF'])
    
    # Obter o conjunto de todas as variáveis únicas das duas colunas
    all_vars = set(clauses_1.keys()).union(set(clauses_2.keys()))
    
    # Criar o vetor de duplas
    pairs = [(clauses_1.get(var, ('', '', '')), clauses_2.get(var, ('', '', ''))) for var in all_vars]
    
    return pairs


# Aplicar a função para criar o vetor de duplas para cada linha do DataFrame
df['Pairs'] = df.apply(create_pairs, axis=1)

# Exibir o DataFrame com os vetores de duplas
df[['Desired Then DNF', 'Shared Then DNF', 'Pairs']]


,Desired Then DNF,Shared Then DNF,Pairs
0,height >= 30 AND source_distance == 0,height >= 30 AND source_distance == 0,"[((source_distance, ==, 0.0), (source_distance..."
1,height >= 30 AND source_distance == 0,height >= 25 AND source_distance == 0,"[((source_distance, ==, 0.0), (source_distance..."
2,height >= 30 AND source_distance == 0,height >= 50 AND source_distance == 0,"[((source_distance, ==, 0.0), (source_distance..."
3,height >= 30 AND source_distance == 0,height <= 100 AND source_distance == 0,"[((source_distance, ==, 0.0), (source_distance..."
4,height >= 30 AND source_distance == 0,height >= 0 AND source_distance == 0,"[((source_distance, ==, 0.0), (source_distance..."
...,...,...,...
127,height >= 30 AND source_distance == 0,height <= 25 AND source_distance >= 75,"[((source_distance, ==, 0.0), (source_distance..."
128,height >= 30 AND source_distance == 0,height <= 25 AND source_distance >= 100,"[((source_distance, ==, 0.0), (source_distance..."
129,height >= 30 AND source_distance == 0,height <= 25 AND source_distance <= 0,"[((source_distance, ==, 0.0), (source_distance..."
130,height >= 30 AND source_distance == 0,height >= 100 AND source_distance >= 75,"[((source_distance, ==, 0.0), (source_distance..."


In [20]:
df[["Pairs"]].iloc[0]

Pairs    [((source_distance, ==, 0.0), (source_distance...
Name: 0, dtype: object

In [21]:
# Carregando os limites das variáveis a partir do JSON
with open('res/variables_config.json', 'r') as file:
    variable_configs = json.load(file)

# Criar um dicionário com os limites de cada variável
limits = {var['name']: {'min': var['min_value'], 'max': var['max_value']} for var in variable_configs}

# Função para calcular o intervalo baseado no operador
def calculate_interval(var, op, val):
    min_limit, max_limit = limits[var]['min'], limits[var]['max']
    
    if op == '>=':
        return (val, max_limit)
    elif op == '<=':
        return (min_limit, val)
    elif op == '>':
        return (val, max_limit)
    elif op == '<':
        return (min_limit, val)
    elif op == '==':
        return (val, val)
    elif op == '!=':
        return (min_limit, max_limit)  # assume o escopo completo
    else:
        raise ValueError(f"Operador '{op}' não reconhecido.")

# Função para calcular o erro característico entre duas tuplas
def calculate_scope_error(pair):
    tuple1, tuple2 = pair
    
    if tuple1[0] != tuple2[0]:  # Verifica se as variáveis são diferentes
        return f"Erro: Variáveis diferentes {tuple1[0]} e {tuple2[0]}"
    
    var = tuple1[0]
    interval1 = calculate_interval(*tuple1)
    interval2 = calculate_interval(*tuple2)
    
    # Calculando o erro característico
    scope_error = (abs(interval1[0] - interval2[0]), abs(interval1[1] - interval2[1]))
    return scope_error

# Exemplo de DataFrame com pares
# data = {
#     'Pairs': [[(('battery', '>=', 10.0), ('battery', '<=', 50.0))],
#               [(('satellite_count', '>=', 5), ('satellite_count', '<=', 10))],
#               [(('wind_speed', '>', 20.0), ('wind_speed', '<', 30.0))]]
# }

# df = pd.DataFrame(data)

# Aplicar a função para calcular o erro característico para cada par de tuplas em 'Pairs'
df['Scope Error by var'] = df['Pairs'].apply(lambda pairs: [calculate_scope_error(pair) for pair in pairs])

# Exibir o DataFrame com os erros característicos
df[['Pairs', 'Scope Error by var']].head()

,Pairs,Scope Error by var
0,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (0.0, 0.0)]"
1,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (5.0, 0.0)]"
2,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (20.0, 0.0)]"
3,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]"
4,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]"


In [22]:
# Função para normalizar o erro com base nos limites da variável
def normalize_error(error, variable):
    if variable in limits:
        min_value = limits[variable]['min']
        max_value = limits[variable]['max']
        # Normaliza o erro usando o intervalo da variável
        return error / (max_value - min_value) if max_value - min_value != 0 else 0
    else:
        return error  # Se a variável não estiver no JSON, retorna o erro sem normalização

# Função para normalizar os erros no 'Scope Error by var' usando a variável identificada nos pares
def normalize_scope_error(pairs, errors):
    normalized_errors = []
    for i, pair in enumerate(pairs):
        variable = pair[0][0]  # Identifica a variável a partir do primeiro elemento da tupla do par
        error = errors[i]
        # Normaliza cada erro dentro da lista de erros associados ao par de variáveis
        normalized_error = [normalize_error(e, variable) for e in error]
        normalized_errors.append(normalized_error)
    return normalized_errors

In [23]:
df['Normalized Scope Error by var'] = df.apply(
    lambda row: normalize_scope_error(row['Pairs'], row['Scope Error by var']), axis=1
)

# Exibir o DataFrame com os erros normalizados
df[['Pairs', 'Scope Error by var', 'Normalized Scope Error by var']].head()

,Pairs,Scope Error by var,Normalized Scope Error by var
0,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]"
1,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (5.0, 0.0)]","[[0.0, 0.0], [0.05, 0.0]]"
2,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (20.0, 0.0)]","[[0.0, 0.0], [0.2, 0.0]]"
3,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]","[[0.0, 0.0], [0.3, 0.0]]"
4,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]","[[0.0, 0.0], [0.3, 0.0]]"


In [24]:
def sum_normalized_errors(normalized_errors):
    # Flatten a lista de listas e soma todos os valores
    return sum(sum(error) for error in normalized_errors)

# Calcular o erro escalar somando todos os valores normalizados
df['Global Normalized Error Sum'] = df['Normalized Scope Error by var'].apply(sum_normalized_errors)

# Exibir o DataFrame com o valor escalar do erro global normalizado
df[['Pairs', 'Scope Error by var', 'Normalized Scope Error by var', 'Global Normalized Error Sum']]

,Pairs,Scope Error by var,Normalized Scope Error by var,Global Normalized Error Sum
0,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (0.0, 0.0)]","[[0.0, 0.0], [0.0, 0.0]]",0.00
1,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (5.0, 0.0)]","[[0.0, 0.0], [0.05, 0.0]]",0.05
2,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (20.0, 0.0)]","[[0.0, 0.0], [0.2, 0.0]]",0.20
3,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]","[[0.0, 0.0], [0.3, 0.0]]",0.30
4,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 0.0)]","[[0.0, 0.0], [0.3, 0.0]]",0.30
...,...,...,...,...
127,"[((source_distance, ==, 0.0), (source_distance...","[(75.0, 100.0), (30.0, 75.0)]","[[0.75, 1.0], [0.3, 0.75]]",2.80
128,"[((source_distance, ==, 0.0), (source_distance...","[(100.0, 100.0), (30.0, 75.0)]","[[1.0, 1.0], [0.3, 0.75]]",3.05
129,"[((source_distance, ==, 0.0), (source_distance...","[(0.0, 0.0), (30.0, 75.0)]","[[0.0, 0.0], [0.3, 0.75]]",1.05
130,"[((source_distance, ==, 0.0), (source_distance...","[(75.0, 100.0), (70.0, 0.0)]","[[0.75, 1.0], [0.7, 0.0]]",2.45


In [25]:
df[['Global Normalized Error Sum']].describe()

,Global Normalized Error Sum
count,132.000000
mean,1.467424
std,0.766224
min,0.000000
25%,0.950000
50%,1.450000
75%,2.050000
max,3.300000


In [26]:
df[['Global Normalized Error Sum','Scenario Similarity']]

,Global Normalized Error Sum,Scenario Similarity
0,0.00,1.000000
1,0.05,0.988888
2,0.20,0.952382
3,0.30,0.950000
4,0.30,0.950000
...,...,...
127,2.80,0.666667
128,3.05,0.666667
129,1.05,0.666667
130,2.45,0.666667


In [28]:
# Criar o scatter plot usando o Plotly
fig = px.scatter(
    df,
    x='Global Normalized Error Sum',
    y='Scenario Similarity',
    title='Accumulated distance from Then vs Scenario Similarity',
    labels={
        'Global Normalized Error Sum': 'Accumulated distance',
        'Scenario Similarity': 'Scenario Similarity'
    },
    template='plotly_white'
)

# Exibir o gráfico
fig.show()

Tentar entender pq no maximo foi 4, como eram 3 variáveis formando 6 [chão, tento], imaginei que no pior dos casos, todos os erros de cada um iria tentar a 1, ou seja [1,1], [1,1], [1,1], logo, somando daria 6. Mas no caso foi 4. Tlz nas condicoes nao trazem situacoes desfavorável para todas as variaveis.